# Importing required libraries

In [415]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN

# Reading in the data

In [416]:
path = "data/preprocessed_data/csv_without_empty_cols_and_merged/"

In [417]:
path = "data/preprocessed_data/csv_without_empty_cols_and_merged/"

df_tosa = pd.read_csv(path+"cof_tosa_aligner.csv", index_col=0, na_values=[-9999, -99999])
df_dctest = pd.read_csv(path+"cof_dctest.csv", index_col=0, na_values=[-9999, -99999], low_memory=False)
df_final = pd.read_csv(path+"final.csv", index_col=0, na_values=[-9999, -99999])


# Taking care of OsaSerialNum duplicates (DCTest)

In [418]:
# df_dctest.sort_values(by=["OsaSerialNum", "TestTimeStamp"], inplace=True)

In [419]:
# df_dctest.drop_duplicates(subset="OsaSerialNum", keep="last", inplace=True)

# Taking care of Containername duplicates (Tosa)

In [420]:
# df_tosa.sort_values(by=["Containername", "TestTimeStamp"], inplace=True)

In [421]:
# df_tosa.drop_duplicates(subset="Containername", keep="last", inplace=True)

# Dropping duplicates and almost empty rows from the output data

In [422]:
df_final.drop_duplicates(inplace=True)

# Instead of removing all rows that do not reach a certain threshhold of non-missing values, like we
# did in the other excercise before, I just dropped all rows that miss the value for "ModuleTxCalPower_dBm".
# This will remove the columns where all the output data are missing (but possibly also some other rows, if we
# are unlucky).
df_final.dropna(subset=["ModuleTxCalPower_dBm"], inplace=True)

# Merging the DataFrames

In [423]:
df_input = df_tosa.merge(df_dctest,
                         left_on="Containername",
                         right_on="OsaSerialNum",
                         suffixes=("_tosa", "_dctest"))

print(df_tosa.shape)
print(df_dctest.shape)
print(df_input.shape)

(33746, 32)
(33967, 84)
(37839, 116)


In [424]:
df_traceability = pd.read_excel("data/original_data/Linkage Map and Traceability.xlsx", "Traceability")

In [425]:
df_input = df_input.merge(df_traceability,
                          on="Component")

In [426]:
df_input.shape

(37839, 117)

In [427]:
df = df_input.merge(df_final,
                    left_on="ToContainer",
                    right_on="ModuleSerialNum",
                    suffixes=("","_final"))

# Removing ID columns

In [428]:
ids = list(df.filter(like="Id").columns)
ids += list(df.filter(like="HistoryID").columns)
print(ids)

df.drop(columns=ids, inplace=True)

['CDOTypeId_tosa', 'DataCollectionDefId_tosa', 'dc_COF_TOSA_AlignerHistoryId', 'HistoryId_tosa', 'HistoryMainlineId_tosa', 'TxnId_tosa', 'CDOTypeId_dctest', 'DataCollectionDefId_dctest', 'dc_COF_DCTestHistoryId', 'HistoryId_dctest', 'HistoryMainlineId_dctest', 'TxnId_dctest', 'dce_HistoryID_tosa', 'ParentHistoryID_tosa', 'dce_HistoryID_dctest', 'ParentHistoryID_dctest']


# Removing columns without variance

In [429]:
# Dropping columns without variance
dropped_columns = []

for c in df.columns:
    count_of_unique_values = len(df[c].dropna().unique())
    if count_of_unique_values == 1:      
        df.drop(columns=c, inplace=True)
        dropped_columns.append(c)
        
print("Columns without variance: %s" % dropped_columns)

df.shape

Columns without variance: ['EngrDataDefID_tosa', 'ParentTableID_tosa', 'ChangeCount_tosa', 'PartNum', 'TestOperator_tosa', 'WhichTest_tosa', 'EngrDataDefID_dctest', 'ParentTableID_dctest', 'FXT_VCSL', 'Wavelength_nm', 'BaseAssemblyNum', 'BasePartNum', 'ChangeCount_dctest', 'IsRetest', 'OsaPartNum', 'Rework', 'TestPlanDate', 'TestPlanName', 'TestPlanVersion', 'WhichTest_dctest']


(37786, 97)

In [430]:
df.to_csv("data_for_modelling.csv")

# Preparing input

In [431]:
# Getting all values of ErrAbbr that occured at least 100 times
df = df[df.groupby("ErrAbbr")["ErrAbbr"].transform(len) > 100]

# Filtering for all rows that do not contain "PASS"
df = df[(df["ErrAbbr"] != "PASS")]

In [432]:
df["ErrAbbr"].value_counts()

-1732    236
TxCal    206
LoPwr    159
RxMsk    146
Name: ErrAbbr, dtype: int64

In [433]:
X = df.drop(columns=df_final.columns)

## Estimating missing input data

In [434]:
# Handling missing values
X.dropna(axis=1, how="all", inplace=True)
X.fillna(X.median(), inplace=True)

## Transforming the categorical variables

In [435]:
categorical_variables = ["TestStation_tosa", "TestStation_dctest", "Site", "Att_bins", "SlotNum"]

# Transform categorical_variables to binary dummy variables
X = pd.get_dummies(X, columns=categorical_variables)

# Remove all remaining categorical columns
X = X.select_dtypes(exclude=object)

In [436]:
with pd.option_context('display.max_columns', 150):
    display(X.head())

,LDPWR_mWs,AliPwr_mWs,CurePwr_mWs,ITH_mA,Slope_Eff,LD_Res,LD_Volt,CALF_COMP,CALF_CONN,T_PO_W,ID_PD_A,BVR_PD_V,BOND_A,BVR_LD_A,SNP_BK,PO_LD_W,VF_LD_V,PO_PCT,SLOPE_WperA,KINK,CORREL,THOLD_A,ROLL,I_NOM_A,IL_PD_A,R_LD_ohm,TRACK8_AperW,TRACK10_AperW,TRACK12_AperW,TRACK15_AperW,DELTRK,PO_H_W,PO_L_W,KINK1,KINK2,WIGGLE_dB,FFT,KINKMULT1,IMAX_A,KINKMULT2,PMAX_W,KINK2BETTER,VI_MASK,VI_KINK,PO_PCT2,T_PO_dBm,PO_LD_dBm,BMRipStd,Wiggle_pMin_W,WIGGLE_pMax_W,TestDuration_s,TestResult,TestStation_tosa_AT-ALIGNCOFT-01,TestStation_tosa_AT-ALIGNCOFT-02,TestStation_tosa_AT-ALIGNCOFT-03,TestStation_tosa_AT-ALIGNCOFT-04,TestStation_tosa_AT-ALIGNCOFT-05,TestStation_tosa_AT-ALIGNCOFT-06,TestStation_tosa_AT-ALIGNCOFT-07,TestStation_tosa_AT-ALIGNCOFT-08,TestStation_tosa_AT-ALIGNCOFT-09,TestStation_tosa_AT-ALIGNCOFT-10,TestStation_dctest_IPH_COFT_01,TestStation_dctest_IPH_COFT_02,TestStation_dctest_IPH_COFT_03,TestStation_dctest_IPH_COFT_04,TestStation_dctest_IPH_COFT_05,TestStation_dctest_IPH_COFT_06,TestStation_dctest_IPH_COFT_07,TestStation_dctest_IPH_COFT_08,TestStation_dctest_IPH_COFT_09,Site_0,Site_1,Site_2,Site_3,Site_4,Site_5,Site_6,Site_7,Site_8,Site_9,Att_bins_-1,Att_bins_0,Att_bins_1,Att_bins_2,Att_bins_3,Att_bins_4,Att_bins_5,SlotNum_0.0,SlotNum_1.0,SlotNum_2.0,SlotNum_3.0,SlotNum_4.0,SlotNum_5.0,SlotNum_6.0,SlotNum_7.0,SlotNum_8.0,SlotNum_9.0
13,3.3820,0.5775,0.5660,0.0,0.0,0.0,0.0,2.0401,1.9964,0.000402,2.131228e-09,31.005,2.085000e-08,1.208360e-09,0.0,0.000337,2.076523,83.72990,0.047092,0.557364,0.999624,0.001061,-2.528080,0.0135,0.000552,52.78397,1.713896,1.615484,1.562788,1.521265,0.941677,0.000536,0.000484,50.51240,233.0817,0.126665,0.169505,129.91130,0.0200,2.113647,0.000774,6.166465,0.0,5.769901,83.06193,-3.956720,-4.727914,0.295899,0.000334,0.000344,22,-10293,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
14,3.3820,0.5775,0.5660,0.0,0.0,0.0,0.0,2.0071,2.0109,0.000603,7.796370e-10,31.001,3.990000e-09,4.498360e-10,0.0,0.000536,2.080706,88.89952,0.068550,0.689251,0.999669,0.000910,-4.386859,0.0089,0.000551,53.26843,1.025409,1.035641,1.027327,1.019694,0.984602,0.000806,0.000729,26.26116,173.4647,0.083197,0.363242,119.56060,0.0199,2.077588,0.001058,5.800860,0.0,5.896650,87.46564,-2.194358,-2.705364,0.261453,0.000528,0.000538,25,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
27,3.3765,0.0000,0.0000,0.0,0.0,0.0,0.0,2.0401,1.9964,0.000621,3.127711e-09,31.011,1.619000e-08,6.474027e-10,0.0,0.000561,2.075640,90.38322,0.071896,0.765628,0.999524,0.000916,-5.415378,0.0085,0.000531,52.64173,0.969051,0.941226,0.962550,0.958889,1.018765,0.000833,0.000767,28.24946,100.4635,0.107034,0.368255,76.91773,0.0200,1.886027,0.001087,5.804416,0.0,5.778452,88.24277,-2.071950,-2.511072,0.408408,0.000548,0.000561,22,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
28,3.3765,0.5716,0.5603,0.0,0.0,0.0,0.0,2.0401,1.9964,0.000621,3.127711e-09,31.011,1.619000e-08,6.474027e-10,0.0,0.000561,2.075640,90.38322,0.071896,0.765628,0.999524,0.000916,-5.415378,0.0085,0.000531,52.64173,0.969051,0.941226,0.962550,0.958889,1.018765,0.000833,0.000767,28.24946,100.4635,0.107034,0.368255,76.91773,0.0200,1.886027,0.001087,5.804416,0.0,5.778452,88.24277,-2.071950,-2.511072,0.408408,0.000548,0.000561,22,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
103,3.3221,0.5672,0.5653,0.0,0.0,0.0,0.0,2.0352,2.0120,0.000632,2.196320e-09,31.004,1.854000e-08,3.018457e-10,0.0,0.000564,2.092213,89.36993,0.072621,0.633654,0.999375,0.000877,-5.383078,0.0084,0.000567,54.28913,1.021971,0.993983,1.018520,0.987671,0.993650,0.000841,0.000769,28.81562,435.9929,0.141585,0.471527,276.26880,0.0200,2.441332,0.001066,6.380777,0.0,5.758220,86.92415,-1.995694,-2.483780,0.449284,0.000549,0.000567,22,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


# t-SNE (Dimensionality reduction)

In [437]:
from sklearn.manifold import TSNE

In [438]:
X_2d = TSNE(perplexity=25, verbose=10).fit_transform(X)

[t-SNE] Computing 76 nearest neighbors...
[t-SNE] Indexed 747 samples in 0.003s...
[t-SNE] Computed neighbors for 747 samples in 0.074s...
[t-SNE] Computed conditional probabilities for sample 747 / 747
[t-SNE] Mean sigma: 9.321415
[t-SNE] Computed conditional probabilities in 0.045s
[t-SNE] Iteration 50: error = 58.1109390, gradient norm = 0.4090248 (50 iterations in 0.820s)
[t-SNE] Iteration 100: error = 53.7336617, gradient norm = 0.3861134 (50 iterations in 0.614s)
[t-SNE] Iteration 150: error = 53.1771927, gradient norm = 0.3829151 (50 iterations in 0.591s)
[t-SNE] Iteration 200: error = 52.4740562, gradient norm = 0.3744095 (50 iterations in 0.602s)
[t-SNE] Iteration 250: error = 52.0477295, gradient norm = 0.3728378 (50 iterations in 0.589s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 52.047729
[t-SNE] Iteration 300: error = 0.4700342, gradient norm = 0.0020255 (50 iterations in 0.553s)
[t-SNE] Iteration 350: error = 0.3851248, gradient norm = 0.0003768 (

In [439]:
df_tsne = pd.DataFrame(X_2d, columns=["x", "y"])

In [440]:
df_tsne.head()

,x,y
0,47.342758,10.580793
1,-16.404428,21.963945
2,2.329787,40.350479
3,2.334367,40.361435
4,-26.720213,-12.970805


__Our new two dimensional data has the same amount of rows as our original data!__

In [441]:
df_tsne.shape

(747, 2)

In [442]:
X.shape

(747, 98)

In [443]:
df_tsne["ErrAbbr"] = df["ErrAbbr"].values
df_tsne.head()

,x,y,ErrAbbr
0,47.342758,10.580793,RxMsk
1,-16.404428,21.963945,RxMsk
2,2.329787,40.350479,-1732
3,2.334367,40.361435,-1732
4,-26.720213,-12.970805,RxMsk


In [444]:
% matplotlib qt

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure()
sns.scatterplot(data=df_tsne, x="x", y="y", hue="ErrAbbr")
#plt.scatter(df_tsne["x"], df_tsne["y"])
plt.show()

# Clustering

In [445]:
from sklearn.cluster import KMeans

## Clustering with KMeans

In [446]:
kmeans = KMeans()

# fitting the data to the clustering algorithm
kmeans.fit(X)

# getting the cluster labels 
labels = kmeans.labels_

In [447]:
pd.Series(labels).value_counts()

5    284
3    231
0    106
1     45
6     35
4     23
7     22
2      1
dtype: int64

## Calculating value counts of the ErrAbbr per cluster

In [448]:
df_cluster_results = X
df_cluster_results["cluster"] = labels
df_cluster_results["ErrAbbr"] = df["ErrAbbr"]

In [449]:
df_cluster_results.groupby("cluster")["ErrAbbr"].value_counts()

cluster  ErrAbbr
0        TxCal      31
         -1732      29
         RxMsk      24
         LoPwr      22
1        LoPwr      14
         TxCal      14
         -1732       9
         RxMsk       8
2        TxCal       1
3        -1732      82
         TxCal      62
         LoPwr      45
         RxMsk      42
4        TxCal      12
         -1732       6
         LoPwr       4
         RxMsk       1
5        -1732      80
         TxCal      73
         RxMsk      67
         LoPwr      64
6        -1732      22
         TxCal       7
         LoPwr       3
         RxMsk       3
7        -1732       8
         LoPwr       7
         TxCal       6
         RxMsk       1
Name: ErrAbbr, dtype: int64